In [1]:
import os
from typing import Any, Dict, List, Tuple
from dotenv import load_dotenv

import psycopg2
from psycopg2.extras import execute_values


def get_pg_conn():
    load_dotenv()
    return psycopg2.connect(
        host=os.getenv("PGHOST"),
        port=int(os.getenv("PGPORT", "5432")),
        dbname=os.getenv("PGDATABASE"),
        user=os.getenv("PGUSER"),
        password=os.getenv("PGPASSWORD"),
    )


def upsert_league_and_seasons(cur, league_info: Dict[str, Any]) -> None:
    # league
    league = league_info.get("league", {})
    country = league_info.get("country", {})
    cur.execute(
        """
        INSERT INTO leagues (id, name, type, logo, country_name, country_code, country_flag)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            name=EXCLUDED.name,
            type=EXCLUDED.type,
            logo=EXCLUDED.logo,
            country_name=EXCLUDED.country_name,
            country_code=EXCLUDED.country_code,
            country_flag=EXCLUDED.country_flag;
        """,
        (
            league.get("id"),
            league.get("name"),
            league.get("type"),
            league.get("logo"),
            country.get("name"),
            country.get("code"),
            country.get("flag"),
        ),
    )

    # seasons
    seasons = league_info.get("seasons") or []
    rows: List[Tuple] = []
    for s in seasons:
        cov = (s.get("coverage") or {})
        fix = (cov.get("fixtures") or {})
        rows.append(
            (
                league.get("id"),
                s.get("year"),
                s.get("start"),
                s.get("end"),
                s.get("current"),
                fix.get("events"),
                fix.get("lineups"),
                fix.get("statistics_fixtures"),
                fix.get("statistics_players"),
                cov.get("injuries"),
                cov.get("odds"),
                cov.get("players"),
                cov.get("predictions"),
                cov.get("standings"),
                cov.get("top_assists"),
                cov.get("top_cards"),
                cov.get("top_scorers"),
            )
        )

    if rows:
        execute_values(
            cur,
            """
            INSERT INTO seasons (
                league_id, year, start_date, end_date, current,
                coverage_fixtures_events, coverage_fixtures_lineups,
                coverage_fixtures_statistics_fixtures, coverage_fixtures_statistics_players,
                coverage_injuries, coverage_odds, coverage_players, coverage_predictions,
                coverage_standings, coverage_top_assists, coverage_top_cards, coverage_top_scorers
            )
            VALUES %s
            ON CONFLICT (league_id, year) DO UPDATE SET
                start_date=EXCLUDED.start_date,
                end_date=EXCLUDED.end_date,
                current=EXCLUDED.current,
                coverage_fixtures_events=EXCLUDED.coverage_fixtures_events,
                coverage_fixtures_lineups=EXCLUDED.coverage_fixtures_lineups,
                coverage_fixtures_statistics_fixtures=EXCLUDED.coverage_fixtures_statistics_fixtures,
                coverage_fixtures_statistics_players=EXCLUDED.coverage_fixtures_statistics_players,
                coverage_injuries=EXCLUDED.coverage_injuries,
                coverage_odds=EXCLUDED.coverage_odds,
                coverage_players=EXCLUDED.coverage_players,
                coverage_predictions=EXCLUDED.coverage_predictions,
                coverage_standings=EXCLUDED.coverage_standings,
                coverage_top_assists=EXCLUDED.coverage_top_assists,
                coverage_top_cards=EXCLUDED.coverage_top_cards,
                coverage_top_scorers=EXCLUDED.coverage_top_scorers;
            """,
            rows,
        )


def upsert_teams_and_venues(cur, teams_resp: List[Dict[str, Any]]) -> None:
    venues_rows: List[Tuple] = []
    teams_rows: List[Tuple] = []

    for item in teams_resp:
        team = item.get("team") or {}
        venue = item.get("venue") or {}

        # venue pode vir null/0
        venue_id = venue.get("id")
        if venue_id:
            venues_rows.append(
                (
                    venue_id,
                    venue.get("name"),
                    venue.get("address"),
                    venue.get("city"),
                    venue.get("capacity"),
                    venue.get("surface"),
                    venue.get("image"),
                )
            )

        teams_rows.append(
            (
                team.get("id"),
                team.get("name"),
                team.get("code"),
                team.get("country"),
                team.get("founded"),
                team.get("national"),
                team.get("logo"),
                venue_id,
            )
        )

    if venues_rows:
        execute_values(
            cur,
            """
            INSERT INTO venues (id, name, address, city, capacity, surface, image)
            VALUES %s
            ON CONFLICT (id) DO UPDATE SET
                name=EXCLUDED.name,
                address=EXCLUDED.address,
                city=EXCLUDED.city,
                capacity=EXCLUDED.capacity,
                surface=EXCLUDED.surface,
                image=EXCLUDED.image;
            """,
            venues_rows,
        )

    if teams_rows:
        execute_values(
            cur,
            """
            INSERT INTO teams (id, name, code, country, founded, national, logo, venue_id)
            VALUES %s
            ON CONFLICT (id) DO UPDATE SET
                name=EXCLUDED.name,
                code=EXCLUDED.code,
                country=EXCLUDED.country,
                founded=EXCLUDED.founded,
                national=EXCLUDED.national,
                logo=EXCLUDED.logo,
                venue_id=EXCLUDED.venue_id;
            """,
            teams_rows,
        )


def upsert_players_and_stats(cur, players_resp: List[Dict[str, Any]]) -> None:
    players_rows: List[Tuple] = []
    stats_rows: List[Tuple] = []

    for item in players_resp:
        player = item.get("player") or {}
        pid = player.get("id")

        birth = player.get("birth") or {}
        players_rows.append(
            (
                pid,
                player.get("firstname"),
                player.get("lastname"),
                player.get("name"),
                player.get("age"),
                player.get("nationality"),
                player.get("height"),
                player.get("weight"),
                player.get("injured"),
                player.get("photo"),
                birth.get("date"),
                birth.get("place"),
                birth.get("country"),
            )
        )

        stats_list = item.get("statistics") or []
        for st in stats_list:
            team = st.get("team") or {}
            league = st.get("league") or {}
            games = st.get("games") or {}
            goals = st.get("goals") or {}
            shots = st.get("shots") or {}
            passes = st.get("passes") or {}
            tackles = st.get("tackles") or {}
            duels = st.get("duels") or {}
            dribbles = st.get("dribbles") or {}
            fouls = st.get("fouls") or {}
            cards = st.get("cards") or {}
            pen = st.get("penalty") or {}
            subs = st.get("substitutes") or {}

            # rating às vezes vem string
            rating = games.get("rating")
            rating_num = None
            try:
                rating_num = float(rating) if rating not in (None, "") else None
            except (TypeError, ValueError):
                rating_num = None

            stats_rows.append(
                (
                    pid,
                    team.get("id"),
                    league.get("id"),
                    league.get("season"),
                    games.get("position"),
                    games.get("number"),
                    games.get("captain"),
                    rating_num,
                    games.get("appearences"),
                    games.get("lineups"),
                    games.get("minutes"),
                    goals.get("total"),
                    goals.get("assists"),
                    goals.get("conceded"),
                    goals.get("saves"),
                    shots.get("total"),
                    shots.get("on"),
                    passes.get("total"),
                    passes.get("key"),
                    passes.get("accuracy"),
                    tackles.get("total"),
                    tackles.get("blocks"),
                    tackles.get("interceptions"),
                    duels.get("total"),
                    duels.get("won"),
                    dribbles.get("attempts"),
                    dribbles.get("success"),
                    dribbles.get("past"),
                    fouls.get("committed"),
                    fouls.get("drawn"),
                    cards.get("yellow"),
                    cards.get("yellowred"),
                    cards.get("red"),
                    pen.get("won"),
                    pen.get("commited"),
                    pen.get("scored"),
                    pen.get("missed"),
                    pen.get("saved"),
                    subs.get("in"),
                    subs.get("out"),
                    subs.get("bench"),
                )
            )

    if players_rows:
        execute_values(
            cur,
            """
            INSERT INTO players (
                id, firstname, lastname, name, age, nationality, height, weight, injured, photo,
                birth_date, birth_place, birth_country
            )
            VALUES %s
            ON CONFLICT (id) DO UPDATE SET
                firstname=EXCLUDED.firstname,
                lastname=EXCLUDED.lastname,
                name=EXCLUDED.name,
                age=EXCLUDED.age,
                nationality=EXCLUDED.nationality,
                height=EXCLUDED.height,
                weight=EXCLUDED.weight,
                injured=EXCLUDED.injured,
                photo=EXCLUDED.photo,
                birth_date=EXCLUDED.birth_date,
                birth_place=EXCLUDED.birth_place,
                birth_country=EXCLUDED.birth_country;
            """,
            players_rows,
        )

    if stats_rows:
        execute_values(
            cur,
            """
            INSERT INTO player_statistics (
                player_id, team_id, league_id, season,
                position, number, captain, rating,
                appearances, lineups, minutes,
                goals_total, goals_assists, goals_conceded, goals_saves,
                shots_total, shots_on,
                passes_total, passes_key, passes_accuracy,
                tackles_total, tackles_blocks, tackles_interceptions,
                duels_total, duels_won,
                dribbles_attempts, dribbles_success, dribbles_past,
                fouls_committed, fouls_drawn,
                cards_yellow, cards_yellowred, cards_red,
                penalty_won, penalty_commited, penalty_scored, penalty_missed, penalty_saved,
                substitutes_in, substitutes_out, substitutes_bench
            )
            VALUES %s
            ON CONFLICT (player_id, team_id, league_id, season) DO UPDATE SET
                position=EXCLUDED.position,
                number=EXCLUDED.number,
                captain=EXCLUDED.captain,
                rating=EXCLUDED.rating,
                appearances=EXCLUDED.appearances,
                lineups=EXCLUDED.lineups,
                minutes=EXCLUDED.minutes,
                goals_total=EXCLUDED.goals_total,
                goals_assists=EXCLUDED.goals_assists,
                goals_conceded=EXCLUDED.goals_conceded,
                goals_saves=EXCLUDED.goals_saves,
                shots_total=EXCLUDED.shots_total,
                shots_on=EXCLUDED.shots_on,
                passes_total=EXCLUDED.passes_total,
                passes_key=EXCLUDED.passes_key,
                passes_accuracy=EXCLUDED.passes_accuracy,
                tackles_total=EXCLUDED.tackles_total,
                tackles_blocks=EXCLUDED.tackles_blocks,
                tackles_interceptions=EXCLUDED.tackles_interceptions,
                duels_total=EXCLUDED.duels_total,
                duels_won=EXCLUDED.duels_won,
                dribbles_attempts=EXCLUDED.dribbles_attempts,
                dribbles_success=EXCLUDED.dribbles_success,
                dribbles_past=EXCLUDED.dribbles_past,
                fouls_committed=EXCLUDED.fouls_committed,
                fouls_drawn=EXCLUDED.fouls_drawn,
                cards_yellow=EXCLUDED.cards_yellow,
                cards_yellowred=EXCLUDED.cards_yellowred,
                cards_red=EXCLUDED.cards_red,
                penalty_won=EXCLUDED.penalty_won,
                penalty_commited=EXCLUDED.penalty_commited,
                penalty_scored=EXCLUDED.penalty_scored,
                penalty_missed=EXCLUDED.penalty_missed,
                penalty_saved=EXCLUDED.penalty_saved,
                substitutes_in=EXCLUDED.substitutes_in,
                substitutes_out=EXCLUDED.substitutes_out,
                substitutes_bench=EXCLUDED.substitutes_bench;
            """,
            stats_rows,
        )

ModuleNotFoundError: No module named 'psycopg2'